In [ ]:
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import l2beat_utils as ltwo
sys.path.pop()

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# # Get token classification
# # https://dune.com/queries/3852319
token_df = d.get_dune_data(query_id = 3852319,
    name = "tokens_by_type_base_opm",
    performance = 'large'#,
    # num_hours_to_rerun=0
)

token_df.sample(5)

In [ ]:
# Get Lastest L2B aoc
l2b_df = ltwo.get_daily_aoc_by_token()
l2b_df_spc = l2b_df[l2b_df['project'].isin(['optimism','base'])]
l2b_df_spc = l2b_df_spc.rename(columns={'project':'blockchain'})

l2b_df_spc = l2b_df_spc[['dt','blockchain','token_type','asset_id','address','usd_value']]
l2b_df_spc['address'] = l2b_df_spc['address'].str.lower()

l2b_df_spc.sample(5)

In [ ]:
cols = ['blockchain','token_address','l1_token','token_classification','symbol']

token_if_l1_df = token_df[cols].rename(columns={'l1_token':'address','token_address':'l2_token'})
token_if_l1_df['l1_token'] = token_if_l1_df['address']
token_if_l2_df = token_df[cols].rename(columns={'token_address':'address'})
token_if_l2_df['l2_token'] = token_if_l2_df['address']

In [ ]:
df_cols = ['blockchain','address','token_classification']
token_if_l1_df = token_if_l1_df[~token_if_l1_df['address'].isna()][df_cols + ['l1_token','l2_token']].drop_duplicates()
token_if_l2_df = token_if_l2_df[~token_if_l2_df['address'].isna()][df_cols + ['l1_token','l2_token']].drop_duplicates()
# token_if_l2_df.sample(5)

In [ ]:
#L2 Mapping preferred
unified_df = l2b_df_spc.merge(token_if_l2_df
                              , on=['blockchain','address'], how='left')
#L1 Mapping fallback
unified_df = unified_df.merge(token_if_l1_df
                              , on=['blockchain','address'], how='left')


In [ ]:
unified_df['token_classification'] = unified_df['token_classification_x'].fillna(unified_df['token_classification_y']).fillna('Unknown')
unified_df['l1_token'] = unified_df['l1_token_x'].fillna(unified_df['l1_token_y'])
unified_df['l2_token'] = unified_df['l2_token_x'].fillna(unified_df['l2_token_y'])

unified_df = unified_df.drop(columns=['token_classification_x', 'token_classification_y','l1_token_x','l1_token_y','l2_token_x','l2_token_y'])

In [ ]:
unified_df['token_classification'] = unified_df.apply(
    lambda row: 'Native ETH' if row['asset_id'] == 'eth-ether' else row['token_classification'], axis=1
)
# Override due to bad Base events
unified_df['token_classification'] = unified_df.apply(
    lambda row: 'Proxy' if row['asset_id'] == 'base:usdc-usd-coin' else row['token_classification'], axis=1
)
# Override OP predeploy
unified_df['token_classification'] = unified_df.apply(
    lambda row: 'Predeploy' if row['asset_id'] == 'op-optimism' else row['token_classification'], axis=1
)

In [ ]:
# Define the priority order for token_classification
priority = {'Proxy': 1, 'Not Proxy or Factory': 2, 'Immutable Factory Created': 3}

# Add a priority column to the DataFrame based on the token_classification
unified_df['priority'] = unified_df['token_classification'].map(priority)

# Sort the DataFrame by the priority column
unified_df = unified_df.sort_values(by='priority')

# Drop duplicates based on the unique columns, keeping the first occurrence (which has the highest priority)
unique_columns = ['dt', 'blockchain', 'token_type', 'asset_id', 'address']
unified_df = unified_df.drop_duplicates(subset=unique_columns, keep='first')

# Drop the priority column as it's no longer needed
unified_df = unified_df.drop(columns=['priority'])

In [ ]:
unified_df.sample(5)

In [ ]:
unified_df.to_csv('csv_outputs/aoc_by_chain_token_classification.csv')

In [ ]:
unified_df[unified_df['address']=='0x833589fcd6edb6e08f4c7c32d4f71b54bda02913']#.sample(5)